# Các mẫu thiết kế GOF : Adapter

**BS. Lê Ngọc Khả Nhi**

# Giới thiệu

Adapter là một mẫu thiết kế thuộc lớp « Cấu trúc », nó cho phép hòa hợp giữa các đối tượng không tương thích với nhau về giao thức, hoặc thích ứng những cấu trúc dữ liệu hoặc giao thức lạ của bên thứ ba, vốn không tương thích với quy trình sẵn có ; mà kết quả sau cùng là người dùng có thể vận hành chương trình bình thường trong mọi hoàn cảnh mà thậm chí không cần ý thức về vấn đề tương thích này.

# Tình huống minh họa :

Trong một dự án nghiên cứu đa trung tâm, dữ liệu của bệnh nhân được tập hợp từ 4 bệnh viện khác nhau, tuy nhiên các bác sĩ ở mỗi nơi lại sử dụng các công cụ khác nhau nên họ đã gửi 4 file dữ liệu với 4 định dạng khác nhau:

+ Bệnh viện A dùng định dạng CSV;

+ Bệnh viện B dùng Excel để nhập liệu, họ gửi định dạng .xlsx;

+ Bệnh viện C dùng SPSS, nên họ gửi dữ liệu định dạng .sav;

+ Bệnh viện D làm việc với R, họ gửi dữ liệu .RData

Chuyên viên lập trình của dự án được yêu cầu phải viết 1 chương trình bằng ngôn ngữ Python cho phép một bác sĩ chỉ cần thi hành 1 method duy nhất, thí dụ data_load() là có thể tải được cả 4 file dữ liệu và ghép nối chúng lại thành 1 Dataframe duy nhất.

In [1]:
import os
import re

Trong thư mục hiện hành có 4 file dữ liệu trên, Nhi viết 1 hàm cho phép xuất ra 1 list gồm tất cả file thuộc các định dạng csv, txt, xlsx, sav và Rdata:

In [3]:
def get_files(path):
    pattern = re.compile(r'\.(txt|csv|xlsx|sav|RData)$')
    files = os.listdir(path)
    return list(filter(pattern.search, files))

files = get_files(os.getcwd())

files

['Clinic_A.txt', 'Clinic_B.xlsx', 'Clinic_C.sav', 'Clinic_D.RData']


# Giải quyết vấn đề bằng Adapter

Đây là một thí dụ đơn giản cho ý tưởng về Adapter. Đối với người làm thống kê, họ thường chỉ quen làm việc với định dạng dữ liệu phổ thông như .csv thông qua cô chế đọc file với Python native, hoặc qua giao thức pandas; có thể xem đây là 1 quy trình mặc định/cơ bản. Khi gặp tình huống định dạng lạ, thí dụ Excel, SPSS, RData, chúng tạo ra trở ngại vì không tương thích với quy trình hiện hành. Trong trường hợp RData, thậm chí nó đòi hỏi một giao thức mới là thư viện pyreadr. 

Như vậy, lập trình viên sẽ phải tạo ra một cấu trúc có tính năng tương thích giữa quy trình mặc định và dữ liệu lạ, hoặc với giao thức lạ từ bên ngoài, để cho người dùng có thể tiếp tục dùng 1 method hiện hành, thí dụ load_data() mà không phải bận tâm về những nhân tố kì lạ từ bên ngoài lẫn bên trong. Điều này gần giống như khi bạn dùng 1 đầu chuyển đổi để cắm các định dạng USB khác nhau với máy tính/điện thoại của mình.

Cấu trúc adapter này có thể là Class hoặc hàm. Trong bài này Nhi sẽ đưa ra 5 giải pháp khác nhau, lần lượt như sau:

## Class Adapter sử dụng multiple inheritance

Giải pháp này sử dụng thiết kế Mix-in và Multiple inheritance; ta sẽ tạo ra 4 class khác nhau lần lượt là CSV, XLSX, SPSS và RDATA, chúng rất đơn giản, chỉ gồm 1 method duy nhất tương ứng với việc tải các định dạng file khác nhau:

method pd.read_csv để tải file .csv hoặc .txt;

method pd.read_excel để tải file .xlsx;

method pd.read_spss để tải file .sav;

method pyreadr.read_r để tải file .RData, sau đó lấy dataframe df bên trong

In [17]:
import pandas as pd
import pyreadr

class CSV:
    """Quy trình mặc định tải file CSV"""

    def load_csv(self, f_name):
        return pd.read_csv(f_name)


class XLSX:
    """Quy trình tải file Excel"""

    def load_excel(self, f_name: str):
        return pd.read_excel(f_name)


class SPSS:
    """Quy trình tải file SPSS"""

    def load_spss(self, f_name):
        return pd.read_spss(f_name)


class RDATA:
    """Quy trình tải file RData"""

    def load_rdata(self, f_name):
        return pyreadr.read_r(f_name)["df"]

Tiếp theo, Nhi tạo 1 class adapter tên là Multi_Loader, và dùng cơ chế multiple inheritance để nó tập hợp cả 4 class CSV, XLSX, SPSS và RDATA bên trên, lúc này cả 4 method chuyên biệt trên đều nằm trong class Multi_Loader, và có thể gọi từ self.spec_method(). Ta chỉ còn việc viết method load() cho phép phân phối từng quy trình đến định dạng file tương ứng.

Như vậy người dùng chỉ cần dùng 1 method load() này là có thể tải cả 4 định dạng khác nhau.

In [18]:
# Adapter class sử dụng multiple Inheritance
class Multi_Loader(CSV, SPSS, XLSX, RDATA):
    """Class tải dữ liệu, dùng 1 method load cho cả 3 định dạng"""

    def load(self, f_name):

        ext = os.path.splitext(f_name)[1]

        if (ext == ".csv") or (ext == ".txt"):
            return self.load_csv(f_name)
        elif ext == ".xlsx":
            return self.load_excel(f_name)
        elif ext == ".sav":
            return self.load_spss(f_name)
        elif ext == ".RData":
            return self.load_rdata(f_name)
        else:
            raise Exception(f"Không hỗ trợ định dạng {ext}")

Chúng ta kiểm tra lại:

In [19]:
comb = pd.DataFrame()

col_names =  ['Age','Sex','CP','BPS','Chol','FBS',
              'ECG','Thalach','Exang','OldPeak','Slope','CA','Thal','Class']
                             
loader = Multi_Loader()

for fi in files:
    df = loader.load(f_name = fi)
    df.columns = col_names
    
    comb = pd.concat([comb, df])
    
comb

,Age,Sex,CP,BPS,Chol,FBS,ECG,Thalach,Exang,OldPeak,Slope,CA,Thal,Class
0,63.0,1.0,4.0,10.0,60.0,2.0,1.0,12.0,3.0,11.0,3.0,1,1,3.0
1,44.0,1.0,4.0,3.0,19.0,2.0,1.0,8.0,2.0,14.0,1.0,1,1,1.0
2,60.0,1.0,4.0,5.0,27.0,2.0,1.0,19.0,3.0,6.0,4.0,1,1,3.0
3,55.0,1.0,4.0,11.0,39.0,2.0,1.0,25.0,3.0,10.0,2.0,1,1,2.0
4,66.0,1.0,3.0,33.0,22.0,3.0,2.0,53.0,3.0,5.0,3.0,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,61.0,0.0,4.0,145.0,307.0,0.0,2.0,146.0,1.0,1.0,2.0,0,7.0,1.0
120,67.0,1.0,1.0,11.0,64.0,3.0,0.0,6.0,2.0,10.0,2.0,1,1.0,4.0
121,53.0,1.0,4.0,123.0,282.0,0.0,0.0,95.0,1.0,2.0,2.0,2,7.0,3.0
122,42.0,1.0,1.0,148.0,244.0,0.0,2.0,178.0,0.0,0.8,1.0,2,3.0,0.0


Kết quả thành công ! Ta đã load cả 4 file với 4 định dạng khác nhau trong list và ghép chúng lại thành dataframe comb

## Class adapter sử dụng composition

Trong giải pháp trên, ta đã tạo ra sự thích nghi bằng cách đưa 4 method khác nhau vào class Adapter. Như ta biết, ta có thể can thiệp vào cấu trúc của class ở 2 cấp độ: fields (thuộc tính) hoặc methods. Trong giải pháp tiếp theo, ta muốn đưa hành vi mới vào class thông qua fields, và chỉ trong khi sử dụng, bằng cơ chế Compositions.

Đầu tiên, Nhi tạo ra 4 class khác nhau lần lượt là CSV, XLSX, SPSS và RDATA, chúng đều có 1 method cùng tên là load(), nhưng nội dung bên trong khác nhau, tương thích với 4 định dạng files.

In [10]:
import pandas as pd
from typing import Optional
import pyreadr

class CSV:
    """Quy trình mặc định tải file CSV"""

    def load(self, f_name):
        return pd.read_csv(f_name)


class XLSX:
    """Quy trình tải file Excel"""

    def load(self, f_name):
        return pd.read_excel(f_name)


class SPSS:
    """Quy trình tải file SPSS"""

    def load(self, f_name):
        return pd.read_spss(f_name)


class RDATA:
    """Quy trình tải file RData"""

    def load(self, f_name):
        return pyreadr.read_r(f_name)["df"]

Tiếp theo Nhi chuẩn bị 1 dictionary để phân phối object của từng class nêu trên cho định dạng file tương ứng:

In [12]:
# Adapter class sử dụng Composition
F_TYPES = {
    ".txt": CSV(),
    ".csv": CSV(),
    ".xlsx": XLSX(),
    ".sav": SPSS(),
    ".RData": RDATA(),}

Cuối cùng, Nhi tạo adapter class tên là Adapt_Loader, ta có thể khởi tạo nó với 1 argument f_type bất kì, là object của 1 trong 4 class nêu trên), xem đó là quy trình mặc định; Lưu ý: không có cũng được (None).

method spec_load() cho phép dùng quy trình mặc định

Khi gọi method load() bên trong adapter này, nó sẽ đọc file extension và đối chiếu với dictionary bên trên, và gán 1 quy trình mới cho f_type (ngay cả khi ta đã khai báo).

In [15]:
class Adapt_Loader:
    """Class tải dữ liệu, dùng 1 method load cho cả 3 định dạng"""

    def __init__(self,f_type: Optional[CSV] = None):
        self.f_type = f_type
        
    def spec_load(self, f_name: str):
        try:
            return self.f_type.load(f_name)
        except:
            print(f"Lỗi: không thể tải được {f_name}")
    
    def load(self, f_name: str):

        ext = os.path.splitext(f_name)[1]
        self.f_type = F_TYPES[ext]
        
        return self.spec_load(f_name)

Giải pháp này cũng cho ra kết quả thành công:

In [20]:
comb2 = pd.DataFrame()

loader = Adapt_Loader()

for fi in files:
    df = loader.load(f_name = fi)
    df.columns = col_names
    
    comb2 = pd.concat([comb2, df])
    
comb2

,Age,Sex,CP,BPS,Chol,FBS,ECG,Thalach,Exang,OldPeak,Slope,CA,Thal,Class
0,63.0,1.0,4.0,10.0,60.0,2.0,1.0,12.0,3.0,11.0,3.0,1,1,3.0
1,44.0,1.0,4.0,3.0,19.0,2.0,1.0,8.0,2.0,14.0,1.0,1,1,1.0
2,60.0,1.0,4.0,5.0,27.0,2.0,1.0,19.0,3.0,6.0,4.0,1,1,3.0
3,55.0,1.0,4.0,11.0,39.0,2.0,1.0,25.0,3.0,10.0,2.0,1,1,2.0
4,66.0,1.0,3.0,33.0,22.0,3.0,2.0,53.0,3.0,5.0,3.0,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,61.0,0.0,4.0,145.0,307.0,0.0,2.0,146.0,1.0,1.0,2.0,0,7.0,1.0
120,67.0,1.0,1.0,11.0,64.0,3.0,0.0,6.0,2.0,10.0,2.0,1,1.0,4.0
121,53.0,1.0,4.0,123.0,282.0,0.0,0.0,95.0,1.0,2.0,2.0,2,7.0,3.0
122,42.0,1.0,1.0,148.0,244.0,0.0,2.0,178.0,0.0,0.8,1.0,2,3.0,0.0


Sơ đồ UML của 2 giải pháp trên như sau:

!['uml'](adapter_UML.png)

Nội dung của dữ liệu và code nằm tại thư mục Github sau:

https://github.com/kinokoberuji/Python-snipets/tree/master/GOF/Adapter

## Giải pháp dùng hàm

Tuy các giáo trình về Design patterns thường sử dụng lập trình hướng đối tượng (OOP), ta hoàn toàn có thể dùng lập trình hàm (function) thay vì OOP (class); trong tình huống này, viết hàm và những công cụ mà Python cung cấp sẵn như Dictionary có thể là giải pháp đơn giản và "Pythonic" hơn rất nhiều.

Đầu tiên, ta cần 1 hàm rời để tải dữ liệu RData (vì đây là method duy nhất không xuất ra trực tiếp dataframe)

Sau đó, ta cần 1 dictionary để phân phối các quy trình tải khác nhau cho từng định dạng file, đây là sự tiện lợi mà Python cung cấp:

In [22]:
def load_rdata(f_name: str) -> pd.DataFrame:
    """Hàm tải file rdata"""
    return pyreadr.read_r(f_name)["df"]

# Sử dụng Python dictionary
LOADERS = {
    ".csv": pd.read_csv,
    ".txt": pd.read_csv,
    ".sav": pd.read_spss,
    ".xlsx": pd.read_excel,
    ".RData": load_rdata,
}

**Class và hàm không khác nhau**

Thực ra, giữa hàm và class trong Python không khác biệt nhau quá xa: khi bạn đặt 1 method --call-- vào class, bạn có thể dùng instance của class đó như 1 hàm.

class Loader sau đây hoạt động như 1 hàm:

In [23]:
class Loader:
    def __call__(self, f_name: str) -> pd.DataFrame:
        ext = os.path.splitext(f_name)[1]
        return LOADERS[ext](f_name)

instance load của class này hoạt động như 1 hàm và thực hiện cùng tính năng mà ta cần:

In [24]:
comb3 = pd.DataFrame()

load = Loader()

for fi in files:
    df = load(f_name = fi)
    df.columns = col_names
    
    comb3 = pd.concat([comb3, df])
    
comb3

,Age,Sex,CP,BPS,Chol,FBS,ECG,Thalach,Exang,OldPeak,Slope,CA,Thal,Class
0,63.0,1.0,4.0,10.0,60.0,2.0,1.0,12.0,3.0,11.0,3.0,1,1,3.0
1,44.0,1.0,4.0,3.0,19.0,2.0,1.0,8.0,2.0,14.0,1.0,1,1,1.0
2,60.0,1.0,4.0,5.0,27.0,2.0,1.0,19.0,3.0,6.0,4.0,1,1,3.0
3,55.0,1.0,4.0,11.0,39.0,2.0,1.0,25.0,3.0,10.0,2.0,1,1,2.0
4,66.0,1.0,3.0,33.0,22.0,3.0,2.0,53.0,3.0,5.0,3.0,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,61.0,0.0,4.0,145.0,307.0,0.0,2.0,146.0,1.0,1.0,2.0,0,7.0,1.0
120,67.0,1.0,1.0,11.0,64.0,3.0,0.0,6.0,2.0,10.0,2.0,1,1.0,4.0
121,53.0,1.0,4.0,123.0,282.0,0.0,0.0,95.0,1.0,2.0,2.0,2,7.0,3.0
122,42.0,1.0,1.0,148.0,244.0,0.0,2.0,178.0,0.0,0.8,1.0,2,3.0,0.0


**Adapter hàm**

Giải pháp thông dụng hơn, là viết 1 hàm đơn giản, thí dụ hàm multi_load sau đây: Nó cũng có tính năng như 1 adapter, tải được cả 4 định dạng file:

In [26]:
# Sử dụng 1 hàm duy nhất 
def multi_load(f_name: str) -> pd.DataFrame:
    """Hàm cho phép tải cả 3 định dạng data"""
    ext = os.path.splitext(f_name)[1]
    try:
        return LOADERS[ext](f_name)
    except:
        print(f"Lỗi: không thể tải file {f_name}")
        return None

In [27]:
comb4 = pd.DataFrame()

for fi in files:
    df = multi_load(f_name = fi)
    df.columns = col_names
    
    comb4 = pd.concat([comb4, df])
    
comb4

,Age,Sex,CP,BPS,Chol,FBS,ECG,Thalach,Exang,OldPeak,Slope,CA,Thal,Class
0,63.0,1.0,4.0,10.0,60.0,2.0,1.0,12.0,3.0,11.0,3.0,1,1,3.0
1,44.0,1.0,4.0,3.0,19.0,2.0,1.0,8.0,2.0,14.0,1.0,1,1,1.0
2,60.0,1.0,4.0,5.0,27.0,2.0,1.0,19.0,3.0,6.0,4.0,1,1,3.0
3,55.0,1.0,4.0,11.0,39.0,2.0,1.0,25.0,3.0,10.0,2.0,1,1,2.0
4,66.0,1.0,3.0,33.0,22.0,3.0,2.0,53.0,3.0,5.0,3.0,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,61.0,0.0,4.0,145.0,307.0,0.0,2.0,146.0,1.0,1.0,2.0,0,7.0,1.0
120,67.0,1.0,1.0,11.0,64.0,3.0,0.0,6.0,2.0,10.0,2.0,1,1.0,4.0
121,53.0,1.0,4.0,123.0,282.0,0.0,0.0,95.0,1.0,2.0,2.0,2,7.0,3.0
122,42.0,1.0,1.0,148.0,244.0,0.0,2.0,178.0,0.0,0.8,1.0,2,3.0,0.0


**Adapter decorator**

Giải pháp cuối cùng mà Nhi đưa ra, đó là sử dụng cơ chế Decorator, như ta biết decorator là 1 hàm bậc cao, nó cho phép thay đổi cả dữ liệu đầu vào, cấu trúc và hành vi của 1 hàm bất kì mà nó áp dụng lên. Giải pháp này được thiết kế như sau

In [28]:
# Sử dụng decorator
def adapt_extension(func):
    def load(**kwargs):
        f_name = kwargs["f_name"]
        ext = os.path.splitext(f_name)[1]
        return LOADERS[ext](f_name)

    return load

@adapt_extension
def load_data(f_name):
    print(f"Đã tải file {f_name}")

adapt_extension là 1 decorator, khi áp dụng lên hàm load_data, nó dùng 1 adaptor bên trong hàm thứ cấp load để tùy biến quy trình tải file theo định dạng trong argument f_name.

Lúc này, hàm load_data từ chỗ trống rỗng không có gì cả, bỗng nhiên có quyền lực mới cho phép tải bất cứ định dạng nào trong 4 định dạng trên.

In [29]:
comb5 = pd.DataFrame()

for fi in files:
    df = load_data(f_name = fi)
    df.columns = col_names
    
    comb5 = pd.concat([comb5, df])
    
comb5

,Age,Sex,CP,BPS,Chol,FBS,ECG,Thalach,Exang,OldPeak,Slope,CA,Thal,Class
0,63.0,1.0,4.0,10.0,60.0,2.0,1.0,12.0,3.0,11.0,3.0,1,1,3.0
1,44.0,1.0,4.0,3.0,19.0,2.0,1.0,8.0,2.0,14.0,1.0,1,1,1.0
2,60.0,1.0,4.0,5.0,27.0,2.0,1.0,19.0,3.0,6.0,4.0,1,1,3.0
3,55.0,1.0,4.0,11.0,39.0,2.0,1.0,25.0,3.0,10.0,2.0,1,1,2.0
4,66.0,1.0,3.0,33.0,22.0,3.0,2.0,53.0,3.0,5.0,3.0,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,61.0,0.0,4.0,145.0,307.0,0.0,2.0,146.0,1.0,1.0,2.0,0,7.0,1.0
120,67.0,1.0,1.0,11.0,64.0,3.0,0.0,6.0,2.0,10.0,2.0,1,1.0,4.0
121,53.0,1.0,4.0,123.0,282.0,0.0,0.0,95.0,1.0,2.0,2.0,2,7.0,3.0
122,42.0,1.0,1.0,148.0,244.0,0.0,2.0,178.0,0.0,0.8,1.0,2,3.0,0.0


# Tổng kết

Adapter là một mẫu thiết kế rất đơn giản nhưng hiệu quả, nó thỏa mãn quy tắc Single Responsibility - tách biệt giao thức và code của người dùng (ngay cả khi bạn dùng hàm thay vì class); quy tắc Open/Closed: cho phép mở rộng tính năng chương trình mà không phải thay đổi nội dung code hiện hành ở cấp độ người dùng. Thí dụ nếu 1 ngày nào đó giao thức pandas, pyreadr không tồn tại nữa, bạn vẫn có thể thích ứng một cách độc lập mà người dùng không hề hay biết; tương tự nếu có một định dạng file mới xuất hiện, bạn chỉ cần thay đổi rất ít cho adapter, người dùng vẫn không hay biết về cơ chế này.

Ta nhận ra điểm tương đồng giữa thiết kế Adapter và Strategy, State (chúng đều có thể triển khai bằng Composition); tuy nhiên các thiết kế kia giải quyết các vấn đề khác, và có thể phức tạp hơn so với adapter;

Ta cũng nhận ra rằng khi học mô thức thiết kế với Python, đôi khi cần nghĩ thoáng và thoát ra khỏi khuôn mẫu OOP của các giáo trình. Đôi lúc Python đã cung cấp sẵn giải pháp đơn giản ngắn gọn hơn nhiều; như dictionary, nametuples, và ta hoàn toàn có thể viết hàm !

Bài thực hành tạm dừng, chúc các bạn học vui và hẹn gặp lại.